In [22]:
import pandas as pd
import tensorflowjs as tfjs
from keras.layers import Dense, Dropout, Flatten
from keras.models import Sequential
from sklearn.model_selection import train_test_split

In [23]:
df = pd.read_csv("../data/candles.csv")

In [24]:
df.head()

,mts,open,close,high,low,volume,pair,timeframe,adx,percentChange,strategy,strategyCode
0,1525938300000,0.79796,0.80054,0.80282,0.79781,1.033720e+06,tXRPUSD,15m,45.007021,0.322282,LONG,1
1,1525939200000,0.80054,0.79978,0.80259,0.79978,1.025322e+05,tXRPUSD,15m,41.418334,-0.095026,SHORT,0
2,1525940100000,0.79977,0.80000,0.80000,0.79858,1.287505e+05,tXRPUSD,15m,35.211220,0.027500,LONG,1
3,1525941000000,0.80000,0.79797,0.80100,0.79782,2.078187e+05,tXRPUSD,15m,32.401693,-0.254396,SHORT,0
4,1525941900000,0.79793,0.79852,0.79860,0.79638,3.543426e+05,tXRPUSD,15m,26.001937,0.068877,LONG,1


In [25]:
model = Sequential()

model.add(Dense(64, input_shape=(2,), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))

In [26]:
model.compile(loss="binary_crossentropy",
              optimizer="Adam",
              metrics=["accuracy"])

In [32]:
X = df[["volume", "adx"]].values
y = df["strategyCode"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [33]:
model.fit(X_train, y_train,
          epochs=50,
          batch_size=20)

Epoch 1/50
329/329 [==============================] - 0s 83us/step - loss: 8.4481 - acc: 0.4711
Epoch 2/50
329/329 [==============================] - 0s 79us/step - loss: 8.2596 - acc: 0.4833
Epoch 3/50
329/329 [==============================] - 0s 78us/step - loss: 8.1600 - acc: 0.4894
Epoch 4/50
329/329 [==============================] - 0s 79us/step - loss: 7.5247 - acc: 0.5289
Epoch 5/50
329/329 [==============================] - 0s 81us/step - loss: 8.4574 - acc: 0.4681
Epoch 6/50
329/329 [==============================] - 0s 79us/step - loss: 7.4177 - acc: 0.5350
Epoch 7/50
329/329 [==============================] - 0s 79us/step - loss: 8.0034 - acc: 0.4985
Epoch 8/50
329/329 [==============================] - 0s 79us/step - loss: 8.0993 - acc: 0.4924
Epoch 9/50
329/329 [==============================] - 0s 79us/step - loss: 8.0993 - acc: 0.4924
Epoch 10/50
329/329 [==============================] - 0s 78us/step - loss: 8.3410 - acc: 0.4772
Epoch 11/50
329/329 [==================

In [34]:
evaluation = model.evaluate(x=X_test, y=y_test, steps=500)

500/500 [==============================] - 0s 514us/step


In [35]:
model.metrics_names

['loss', 'acc']

In [36]:
print(evaluation)

[8.509125709533691, 0.46625766158103943]
